### Inital test of fetch/push pipeline

In [12]:
import requests
import importlib
import blizzard_api
import mysql
import mplusdb
import blizzard_credentials
import pandas as pd
import sys
import time

In [3]:
#grab access token
importlib.reload(blizzard_credentials)
tokens_file_path = '.api_tokens'
authorization = blizzard_credentials.Credentials(tokens_file_path)
access_token = authorization.access_token

In [6]:
#grab list of realm ids from MDB
#conn.close()
mdb = mplusdb.MplusDatabase('.db_config')
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute('SELECT cluster_id, region from realm')
realm_ids_raw = cursor.fetchall()
cursor.execute('SELECT * from dungeon')
dungeon_ids_raw = cursor.fetchall()
cursor.close()
conn.close()

In [7]:
dungeons = dict(dungeon_ids_raw)
print(dungeons)

{197: 'Eye of Azshara', 198: 'Darkheart Thicket', 199: 'Black Rook Hold', 200: 'Halls of Valor', 206: "Neltharion's Lair", 207: 'Vault of the Wardens', 208: 'Maw of Souls', 209: 'The Arcway', 210: 'Court of Stars', 227: 'Return to Karazhan', 233: 'Cathedral of Eternal Night', 234: 'Return to Karazhan', 239: 'Seat of the Triumvirate', 244: "Atal'Dazar", 245: 'Freehold', 246: 'Tol Dagor', 247: 'The MOTHERLODE!!', 248: 'Waycrest Manor', 249: "Kings' Rest", 250: 'Temple of Sethraliss', 251: 'The Underrot', 252: 'Shrine of the Storm', 353: 'Siege of Boralus', 369: 'Operation: Mechagon - Junkyard', 370: 'Operation: Mechagon - Workshop'}


In [8]:
realm_ids = pd.DataFrame(realm_ids_raw,
                         columns = ['cluster_id', 'region'])

In [9]:
region_to_realms = realm_ids.groupby('region')['cluster_id'].agg('unique').apply(list).to_dict()

In [10]:
region_map = {1:'us', 2:'kr', 3:'eu', 4:'tw'}

In [24]:
# loop through time periods / realms / regions
importlib.reload(blizzard_api)

region_encoder = {'us':1, 'kr':2,'tw':4, 'eu':3 }
start = time.time()
mdb = mplusdb.MplusDatabase('.db_config')
i = 0
region_token = 'us'
period = 762
response_tracker = []
call_factory = blizzard_api.CallFactory(access_token, region_token)
for dungeon in dungeons:
    if dungeon < 244: ##### <<<<<<<<
        continue
    for realm in [3685]:#region_to_realms[region_encoder[region_token]]:
        leaderboard_call = call_factory.get_mythic_plus_leaderboard_call(
            dungeon_id = dungeon,
            realm_id = realm,
            period = period)
        response = requests.get(leaderboard_call)
        if response.status_code == 200:
            leaderboard = blizzard_api.KeyRunLeaderboard(response.json())
           
            rosters = leaderboard.concat_rosters_for_sql_insert()
            
            insert_query_rosters = 'INSERT IGNORE INTO roster VALUES ' + ','.join(rosters)
        
            runs = leaderboard.concat_runs_for_sql_insert()
            insert_query_runs = 'INSERT IGNORE INTO run VALUES ' + ','.join(runs)
            conn = mdb.connect()
            cursor = conn.cursor()
            conn.start_transaction()
            
            try:
                cursor.execute(insert_query_runs)
                conn.commit()
                #print('runs ', cursor.rowcount)
                cursor.execute(insert_query_rosters)
                conn.commit()
                #print('rosters', cursor.rowcount)
                response_tracker.append([leaderboard_call, response.status_code, True])
            except:
                print("something went wrong")
                response_tracker.append([leaderboard_call, response.status_code, False])
            finally:
                cursor.close()
                conn.close()
        else:
            response_tracker.append([leaderboard_call, response.status_code, False])
        i += 1
        print('\rquerying board %s out of %s ' % (i, 
            12*len(region_to_realms[region_encoder[region_token]])), end='')
        #if i == 5:
        #    break
    #break
print('\n')

time_elapsed = time.time() - start
print('time elapsed', time_elapsed / 60)

querying board 12 out of 1380 

time elapsed 0.18672302563985188


In [295]:
df = pd.DataFrame(response_tracker, columns = ['call', 'reponse_status_code', 'upload'])
df.head(10)

In [292]:
df.loc[~df.upload, 'call']

Series([], Name: call, dtype: object)